## Multi Source - BFS Parents Algorithm (Julia - Graph Blas Library)


In [3]:
using SuiteSparseGraphBLAS
using SparseArrays
using LinearAlgebra

In [49]:
function insert(P, R, s, n)
    for i = 1:s
        for j = 1:n
            if(P[i, j]!=nothing && R[i, j]==nothing) #*
                R[i, j] = P[i, j] 
            end
        end
    end
    
    return R
    
end

insert (generic function with 2 methods)

In [92]:
#A is the input matrix, s is the source node, and n is the number of nodes
function bfs_parents(A, S, n, s)
     
    index = GBMatrix{Int64}(s, n)
            for i = 1:n
                for j = 1:s
                    index[j, i] = i
                end
            end
    
    #frontier matrix -> F=S source nodes insrted in frontier
    F = GBMatrix{Int64}(n, n)
    F = S
    
    #parent matrix -> P[S]=S because the source is already visited
    P = GBMatrix{Int64}(n, n)
    P = S
    
    #result matrix -> R[S]=0 because the parents of the source are considered node 0
    R = GBMatrix{Int64}(s, n)
    LR = GBMatrix{Int64}(s, n)
    R = copy(S)
    for i = 1:n
        for j = 1:s
            if(R[j, i]!=nothing)
                R[j, i] = 0
            end
        end
    end   
    
    print("STARTING ...\n\n")
    for i = 1:n-1
            F = mul(F, A, Semirings.MIN_FIRST, mask=P, desc=Descriptors.RC)
            P = F[:, :, mask=F, desc=Descriptors.S]
            F = index[:, :, mask=F, desc=Descriptors.S]
            R = insert(P, R, s, n)
            if(R == LR)
                break
            end
            print("\nFRONTIER UPDATED\n\n")
            print(F)
            LR = copy(R)
            print("\n\nITERATION FINISHED\n\n")
            
    end
    
    print("COMPLETED\n\n")
    print("RESULT\n\n")
    print(R)
    
end

bfs_parents (generic function with 1 method)

## Test

In [93]:
S = GBMatrix(sparse([[1, 0, 0] [0, 0, 0] [0, 3, 0] [0, 0, 4] [0, 0, 0] [0, 0, 0] [0, 0, 0]]))
matrix =  GBMatrix(sparse([[0, 0, 0, 1, 0, 0, 0] [1, 0, 0, 0, 0, 0, 0] [0, 0, 0, 1, 0, 1, 1] [1, 0, 0, 0, 0, 0, 1] [0, 1, 0, 0, 0, 0, 1] [0, 0, 1, 0, 1, 0, 0] [0, 1, 0, 0, 0, 0, 0]]))
#print(source')
@time bfs_parents(matrix, S, 7, 3)





STARTING ...


FRONTIER UPDATED

[nothing 2 nothing 4 nothing nothing nothing; nothing nothing nothing nothing nothing 6 nothing; 1 nothing 3 nothing nothing nothing nothing]

ITERATION FINISHED


FRONTIER UPDATED

[1 nothing 3 nothing 5 nothing 7; nothing nothing 3 nothing nothing nothing nothing; nothing 2 nothing 4 nothing 6 nothing]

ITERATION FINISHED


FRONTIER UPDATED

[nothing 2 nothing 4 nothing 6 nothing; nothing nothing nothing nothing nothing 6 nothing; 1 nothing 3 nothing 5 nothing 7]

ITERATION FINISHED

COMPLETED

RESULT

[0 1 4 1 2 3 2; nothing nothing 0 nothing nothing 3 nothing; 4 1 4 0 2 3 2]  0.048975 seconds (52.39 k allocations: 3.000 MiB, 91.29% compilation time)
